In [11]:
import numpy as np

nos = np.array(['0.125691804516', 'MH 20EE 7598]', '"PY OIBB 5956', 'DL 1OCEL58', 'IH2F.036', 'MHOZFE8819', 'TN 87 A 3980]', '6JO5Ja114h', '(L 26 H 5009', 'Tn21AQ1116', "'TSOZFX353L", 'IMHO1av8866', 'DL8CX4850'])
print(nos[1])

MH 20EE 7598]


In [12]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from datetime import datetime
from fuzzywuzzy import fuzz

#Use a service account.
cred = credentials.Certificate("SeviceAccount.json")
app = firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [13]:
def checkdate(data,field):
    if data and data[field]:
        date_obj = datetime.strptime(data[field], "%d-%b-%Y")
        current_date_obj = datetime.now()
        if current_date_obj > date_obj:
            # print('Expired '+field)
            return True
        print(date_obj)

In [14]:
parivahan_car_nos = np.array([])
parivahan_car = db.collection(u'parivahan_car_data')
for doc in parivahan_car.stream():
    parivahan_car_nos = np.append(parivahan_car_nos, doc.id)
    print(doc)
parivahan_car_nos

array(['DL10CE4581', 'DL8CX4850', 'GJ05JA1143', 'KL26H5009', 'MH01AV8866',
       'MH02FE8819', 'MH14DT8831', 'MH20BQ20', 'MH20EE7598', 'MH20EJ0365',
       'PY01BB5956', 'TN21AQ1114', 'TN21AQ1116', 'TN87A3980',
       'TS07FX3534'], dtype='<U32')

In [15]:
nos_cleaned = np.array([])
for no in nos:
    no = no.replace(' ', '')
    no = no.replace('"', '')
    no = no.replace("'", '')
    no = no.replace(']', '')
    no = no.upper()
    
    no_12 = no[0:2]
    no_12 = no_12.replace('(', 'K')
    no_12 = no_12.replace('0', 'O')
    no_12 = no_12.replace('1', 'I')
    no_12 = no_12.replace('5', 'S')
    no_12 = no_12.replace('8', 'B')
    no_12 = no_12.replace('6', 'G')
    
    no_34 = no[2:4]
    no_34 = no_34.replace('I', '1')
    no_34 = no_34.replace('Z', '2')
    no_34 = no_34.replace('O', '0')
    no = no_12 + no_34 + no[4:]
    # print(no)
    for parivahan_car_no in parivahan_car_nos:
        fuzz_score = fuzz.partial_ratio(no, parivahan_car_no)
        
        if(fuzz_score > 70):
            no = parivahan_car_no
            print(fuzz_score)
            break
    
    # search car no on parivahan database
    parivahan_car = db.collection(u'parivahan_car_data').document(no)
    # get car data from parivahan database
    current_car_data = db.collection(u'current_car_data').document(no)
    # set car data to current_car_data
    data = parivahan_car.get().to_dict()
    
    dt = datetime.now()
    time_str = dt.strftime("%I:%M%p")
    # print(data)
    if data:
        data['created_date'] = time_str
        data['validity'] = 'true'
    # current_car_data.set(data)
    nos_cleaned = np.append(nos_cleaned, no)
    # checkdate(data,'tax_validity')
    if checkdate(data,'insurance_validity')==True or checkdate(data,'puc_validity')==True:
        print('Expired')
        invalid_car_data = db.collection(u'invalid_car_data').document(no)
        data['validity'] = 'false'
        invalid_car_data = invalid_car_data.set(data)
    
    if data:   
        current_car_data.set(data)
print(nos_cleaned)

100
2023-12-28 00:00:00
2023-12-28 00:00:00
100
2024-01-06 00:00:00
89
100
Expired
100
2023-10-11 00:00:00
Expired
90
2023-04-30 00:00:00
Expired
100
2023-04-29 00:00:00
2023-10-14 00:00:00
90
2024-03-19 00:00:00
Expired
80
2023-12-20 00:00:00
100
2024-02-13 00:00:00
2023-08-31 00:00:00
100
['O.125691804516' 'MH20EE7598' 'PY01BB5956' 'DL10CE4581' 'IH2F.036'
 'MH02FE8819' 'TN87A3980' 'GJ05JA1143' 'KL26H5009' 'TN21AQ1114'
 'TS07FX3534' 'MH01AV8866' 'DL8CX4850']


In [16]:
dt = datetime.now()
time_str = dt.strftime("%I:%M%p")
data['created_date'] = time_str
time_str

'03:48PM'

In [17]:
for parivahan_car_no in parivahan_car_nos:
    fuzz_score = fuzz.partial_ratio('IMH01AV8866', parivahan_car_no)
    print(fuzz_score)

20
22
20
22
100
50
50
38
40
40
30
20
30
22
10
